In [1]:
import pandas as pd
import pickle 
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import requests
import urllib
import os
import zipfile
import lxml

In [2]:
os.chdir('/work/notebook/')

# 1. Data Download


In [17]:
def download_applications(year):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:57.0) Gecko"\
               "/20100101 Firefox/57.0",}

    lnk = 'https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/'\
          + str(year) + '/'
    rr = requests.get(lnk, headers=headers)
    html = rr.content
    soup = BeautifulSoup(html, "html.parser")
    urls = [x.get('href') for x in soup.find_all('a') 
            if '.zip' in x.get('href')]
    target_dir = '../data/apps/' + str(year) + '/'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    [urllib.request.urlretrieve(lnk + x, 
                                '../data/apps/' + str(year) + '/'+x) for
     x in urls]

In [ ]:
year = 2017
download_applications(year)
olddir = os.getcwd()
newdir = '/work/data/apps/' + str(year) + '/'
os.chdir(newdir)

In [4]:
def unzip_file(fnm):
    zip_ref = zipfile.ZipFile(fnm, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
    return fnm.split('/')[-1].replace('.zip', '.xml')

In [3]:
year = 2017
#download_applications(year)
olddir = os.getcwd()
newdir = '/work/data/apps/' + str(year) + '/'
os.chdir(newdir)

In [10]:
newdir

'/work/data/apps/2017/'

In [14]:
x.split('.')[-1] for x in os.listdir(newdir) if x.split('.')[-1]  == 'zip']

['ipa170105',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa170720',
 'zip',
 'ipa170112',
 'zip',
 'ipa170817',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa170323',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa170209',
 'ipa171102',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa171019',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa171005',
 'zip',
 'zip',
 'zip',
 'zip',
 'zip',
 'ipa170309',
 'ipa170216',
 'zip',
 'zip',
 'zip']

In [15]:
%%time
for fnm in [x for x in os.listdir() if x.split('.')[-1]=='zip']:
    fnm_xml = unzip_file(fnm)
    print(fnm_xml)
    !bash /work/split_apps.sh {fnm_xml}
    !rm {fnm}

ipa170615.xml
ipa170511.xml
ipa170525.xml
ipa170727.xml
ipa170824.xml
ipa171109.xml
ipa170330.xml
ipa170119.xml
ipa170316.xml
ipa170223.xml
ipa170413.xml
ipa170518.xml
ipa171228.xml
ipa170427.xml
ipa170713.xml
ipa170302.xml
ipa170601.xml
ipa170914.xml
ipa171221.xml
ipa170928.xml
ipa170810.xml
ipa170831.xml
ipa170406.xml
ipa170706.xml
ipa170921.xml
ipa170629.xml
ipa170504.xml
ipa170105.xml
mkdir: cannot create directory 'ipa170105': File exists
ipa170622.xml
ipa171123.xml
ipa171026.xml
ipa170202.xml
ipa170126.xml
ipa171116.xml
ipa171214.xml
ipa170608.xml
ipa170803.xml
ipa171207.xml
ipa171130.xml
ipa170420.xml
ipa170907.xml
ipa171012.xml
CPU times: user 2min 31s, sys: 27.7 s, total: 2min 59s
Wall time: 8min 14s


In [ ]:
% rm ipa170727.zip

# 2. Find doc-numbers

In [16]:
def get_docnumbers(fnm):
    #try:
        tree = ET.parse(fnm)
        dc_nums = ["".join([x for x in numb.itertext()]) for numb in 
                                           tree.iter(tag = 'doc-number')]
        return pd.DataFrame.from_dict({fnm.split('/')[-1] : dc_nums}).T
    #except:
    #    return pd.DataFrame.from_dict({fnm.split('/')[-1] : [-1111]}).
    
    
def find_app_ids(file_dir):
    #from IPython.core.debugger import Tracer; Tracer()()
    rows = [get_docnumbers(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    pdrows = pd.concat(rows)
    pdrows.index = file_dir + pdrows.index
    return pdrows

In [17]:
%%time
all_app_ids = pd.concat([find_app_ids(newdir + x +'/') 
                              for x in os.listdir(newdir)])
all_app_ids = all_app_ids.rename(columns={1: 'app_id'})
all_app_ids.to_csv(str(year) + '_app_fnm_and_ids.csv')

CPU times: user 36min 36s, sys: 59.2 s, total: 37min 35s
Wall time: 1h 3min 13s


In [20]:
all_app_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374732 entries, /work/data/apps/2017/ipa171221/F_1722.xml to /work/data/apps/2017/ipa170406/F_2924.xml
Columns: 776 entries, 0 to 775
dtypes: object(776)
memory usage: 2.2+ GB


# 3. Looking at the doc-numbers

In [23]:
all_app_ids.app_id.unique().shape

(374472,)

In [28]:
dupes = all_app_ids.loc[all_app_ids.app_id.duplicated(keep=False)].\
                    sort_values('app_id')

In [30]:
dupes

,0,app_id,2,3,4,5,6,7,8,9,...,766,767,768,769,770,771,772,773,774,775
/work/data/apps/2017/ipa170202/F_2073.xml,20170029169,13671978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170720/F_1749.xml,20170203877,13671978,61557168,20170029169,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170427/F_752.xml,20170112793,14209480,11576871,PCT/DK05/00648,14209480,60691513,20140193495,20160136125,20160271093,20170014367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170119/F_599.xml,20170014367,14209480,11576871,PCT/DK05/00648,14209480,60691513,20140193495,20160136125,20160271093,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170706/F_7177.xml,20170195681,14311083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa171026/F_7523.xml,20170310982,14311083,13431920,8798383,14311083,61465957,20170195681,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170810/F_4335.xml,20170228221,14320634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa171228/F_4385.xml,20170371633,14320634,61841421,20170228221,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa171019/F_4814.xml,20170300526,14336505,20140330864,20160267130,20170147634,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2017/ipa170525/F_4744.xml,20170147634,14336505,20140330864,20160267130,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
